In [1]:
import numpy as np
import sys
from collections import Counter
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
import math
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import contractions
import re
from nltk.corpus import stopwords

In [2]:
STOP_WORDS = set(stopwords.words('english'))

In [3]:
# Reading in dataset
yelp_df = pd.read_csv("yelp.csv")
yelp_df = yelp_df[['review_id', 'text', 'stars']]
yelp_df['stars'] = yelp_df['stars'].apply(str)
yelp_df

,review_id,text,stars
0,fWKvX83p0-ka4JS3dc6E5A,My wife took me here on my birthday for breakf...,5
1,IjZ33sJrzXqU-0X6U8NwyA,I have no idea why some people give bad review...,5
2,IESLBzqUCLdSzSqm0eCSxQ,love the gyro plate. Rice is so good and I als...,4
3,G-WvGaISbqqaMHlNnByodA,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,1uJFq2r5QfJG_6ExMRCaGw,General Manager Scott Petello is a good egg!!!...,5
...,...,...,...
9995,Ubyfp2RSDYW0g7Mbr8N3iA,First visit...Had lunch here today - used my G...,3
9996,2XyIOQKbVFb6uXQdJ0RzlQ,Should be called house of deliciousness!\n\nI ...,4
9997,jyznYkIbpqVmlsZxSDSypA,I recently visited Olive and Ivy for business ...,4
9998,5UKq9WQE1qQbJ0DJbc-B6Q,My nephew just moved to Scottsdale recently so...,2


In [4]:
def apply_space_tokenization(text):
    new_text = text.split(' ')
    return new_text

def apply_lowercase(text):
    new_text = text.lower()
    return new_text

In [5]:
yelp_df['text'] = yelp_df['text'].apply(apply_lowercase)
#yelp_df['text'] = yelp_df['text'].apply(apply_space_tokenization)
yelp_df

,review_id,text,stars
0,fWKvX83p0-ka4JS3dc6E5A,my wife took me here on my birthday for breakf...,5
1,IjZ33sJrzXqU-0X6U8NwyA,i have no idea why some people give bad review...,5
2,IESLBzqUCLdSzSqm0eCSxQ,love the gyro plate. rice is so good and i als...,4
3,G-WvGaISbqqaMHlNnByodA,"rosie, dakota, and i love chaparral dog park!!...",5
4,1uJFq2r5QfJG_6ExMRCaGw,general manager scott petello is a good egg!!!...,5
...,...,...,...
9995,Ubyfp2RSDYW0g7Mbr8N3iA,first visit...had lunch here today - used my g...,3
9996,2XyIOQKbVFb6uXQdJ0RzlQ,should be called house of deliciousness!\n\ni ...,4
9997,jyznYkIbpqVmlsZxSDSypA,i recently visited olive and ivy for business ...,4
9998,5UKq9WQE1qQbJ0DJbc-B6Q,my nephew just moved to scottsdale recently so...,2


In [6]:
# Creating a training testing split of the data
X = yelp_df['text']
y = yelp_df['stars']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)

# Preprocessing Expirements
#### Potentially we do some constants throughout our experiments? I.e Tokenize just by spaces, and lowecase all

### No Pre-Processing

### Lemanization

In [7]:
def apply_lem(text):
    lem = WordNetLemmatizer()
    new_text = []
    for word in text.split(' '):
        new_word = lem.lemmatize(word)
        new_text.append(new_word)
    review = ' '.join(new_text)
    return review

### Stemming

In [8]:
def apply_stemming(text):
    ps = PorterStemmer()
    new_text = []
    for word in text.split(' '):
        new_word = ps.stem()
        new_text.append(new_word)
    review = ' '.join(new_text)
    return review

### Contraction expansion

In [9]:
def apply_contraction_expansion(text):
    new_text = []
    for word in text.split(' '):
        new_word = contractions.fix(text)
        new_text.append(new_word)
    review = ' '.join(new_text)
    return review

### Removing Number related text

In [10]:
def apply_number_removal(text):
    review = re.sub(r'[0-9]', '', text)
    return review

### Punctuation Removal

In [11]:
def apply_punctuation_removal(text):
    review = re.sub(r'[.,!?-_;:&#$@%=+]', '', text)
    return review

### Stopword Removal

In [12]:
def apply_stopword_removal(text):
    new_text = []
    for word in text.split(' '):
        if text in STOP_WORDS:
            continue
        else:
            new_text.append(word)
    review = ' '.join(new_text)
    return review

### Spell Check Removal

### NER

### Removing Certain P.O.S

### Pre-Processing Combination
#### Perhaps we run experiments with random combinations of all methods and see best performance

# Model Creation & Test Execution

# Result Collection & Visualization